
#What are Metric Views?

Metric views in Unity Catalog simplify access to key business KPIs, empowering teams across your organization to make data-driven decisions confidently.

By abstracting away the underlying data complexity, metric views ensure consistent definitions and calculations so that users can focus on insights, not infrastructure. Whether you're in analytics, finance, or operations, accessing trusted metrics has never been easier.

In [0]:
%run ../_resources/00-setup $reset_all_data=false

In [0]:
select CURRENT_CATALOG() as catalog, CURRENT_SCHEMA() as schema

# Create a Metric View with Databricks Assistant

Leveraging the Assistant and UI streamlines the process, making it accessible for technical and non-technical users.

<img src="https://github.com/databricks-demos/dbdemos-resources/raw/main/images/retail/lakehouse-churn/create_mv_assistant.gif" style="float: right" width="1200px"/>

Open the source table and ask the assistant to "give me the metric views definition for this table"`. You can even open the assistant generated YAML in a metric view editor.



The YAML might look something like this:

~~~
version: 0.1

source: demos.dbdemos_retail_c360.churn_users

dimensions:
  - name: Age Group
    expr: age_group

measures:
  - name: Total Users
    expr: COUNT(user_id)
~~~

* **Measures** are aggregations that do not have a predefined aggregation level. Every measure must represent an aggregated result.

* **Dimensions** are categories (like country, product, or date) used to break down your data. They can use any valid SQL formula.

## Explore the Metric View in the Catalog Explorer
- Check Measures and Dimensions
- [Optional] Add a description to the metric view
- [Optional] Add comments/tags
- Click the Permissions tab (UC privileges)
- Check the lineage tab (lineage graph)

## Edit the Metric View
- Click Edit. This will take you to the YAML edit page. Let's enhance the UC metric view to look like this [YAML]($churn_users_mteric_view.yaml).

- Explanation of the fields:
1.  **Filters**: Filters are optional conditions, like the WHERE clause in SQL. Let’s add a filter to the UC metrics view. This filter will apply to all queries inside that view, working like a global filter.
~~~
filter: last_activity_date >= '2020-01-01'
~~~
2. **Join**: A join is a way to connect two tables to combine related information. For example, we join the orders table to get order details and user info.
~~~
joins:
  - name: churn_orders
    source: demos.dbdemos_retail_c360.churn_orders
    using: ["user_id"]
~~~
3. Dimension: A dimension has two things: `name: the column’s nickname` and `expr: the actual column or a formula`.
~~~
 - name: Age Group
   expr: age_group
~~~
4. Dimension with SQL function: We can use functions in the `expr` of dimensions.
~~~
 - name: Last Activity Date
   expr: date_trunc('day', last_activity_date)
~~~
5. Measure with SQL function: A measure has the following components: `Name: The alias of the column` and `Expr: The aggregate SQL expression`.
~~~
 - name: Total Users
   expr: COUNT(DISTINCT user_id)
~~~
6. Measure with filter: A measure-level filter that applies only to queries on this measure can be added. This filter only affects this specific measure and doesn’t change or hide any other data or numbers.
~~~
 - name: Churned Users
   expr: COUNT(DISTINCT user_id) FILTER (WHERE churn = 1)
~~~
7. Measure with window function: Window measures allow you to define metrics that use windowed, cumulative, or semi-additive aggregations. To configure a window function, you need to specify the following attributes: `order (required): The dimension used to determine the ordering of the window`. This must be a dimension exposed by the metric view. `semiadditive (required): Defines how the window measure should be summarized when the order field is not part of the query's group by clause`. You can choose to return either the first or last value in the sequence to enable semiadditive behavior. `range (required): Indicates the extent of the window`.
~~~
 - name: Trailing 30-Day Active Users
   expr: COUNT(DISTINCT user_id)
   window:
      - order: Last Activity Date
        range: trailing 30 day
        semiadditive: last
~~~
8. Measure with other measures: Here’s a simple example of making a new measure by using two existing ones. Remember, you have to use the MEASURE function to refer to the existing measures.
~~~
 - name: Churn Rate Percentage
   expr: (MEASURE(`Churned Users`) / MEASURE(`Total Users`)) * 100
~~~

# Create a Metric View with SQL
We created a metric view from the catalog explorer using the AI assistant in the last steps. Now, we will create a metric view using SQL. The YAML can be provided in the SQL statement itself.

In [0]:

-- Create a Metric View with business-friendly dimensions and measures
CREATE OR REPLACE VIEW churn_users_metric_view
WITH METRICS
LANGUAGE YAML
COMMENT 'Metric view demonstration'
AS $$
version: 0.1
source: demos.dbdemos_retail_c360.churn_users

filter: last_activity_date >= '2020-01-01'

joins:
  - name: churn_orders
    source: demos.dbdemos_retail_c360.churn_orders
    using: ["user_id"]

dimensions:
  - name: Age Group
    expr: age_group

  - name: Canal
    expr: canal

  - name: Country
    expr: country

  - name: Order Creation Date
    expr: churn_orders.creation_date

  - name: Last Activity Date
    expr: date_trunc('day', last_activity_date)

  - name: Gender
    expr: CASE
            WHEN gender = 0 THEN 'Female'
            WHEN gender = 1 THEN 'Male'
            ELSE 'Other'
          END

measures:
  - name: Total Users
    expr: COUNT(DISTINCT user_id)

  - name: Average Age Group
    expr: AVG(age_group)

  - name: Active Users
    expr: COUNT(DISTINCT user_id) FILTER (WHERE last_activity_date > CURRENT_DATE - INTERVAL 3 YEARS)

  - name: Churned Users
    expr: COUNT(DISTINCT user_id) FILTER (WHERE churn = 1)

  - name: Churn Rate
    expr: COUNT(DISTINCT user_id) FILTER (WHERE churn = 1) / COUNT(DISTINCT user_id)

  - name: Churn Rate Percentage
    expr: (MEASURE(`Churned Users`) / MEASURE(`Total Users`)) * 100

  - name: Total Order Amount
    expr: SUM(churn_orders.amount)

  - name: Trailing 30-Day Active Users
    expr: COUNT(DISTINCT user_id)
    window:
      - order: Last Activity Date
        range: trailing 30 day
        semiadditive: last
$$;

## Certify the Metric View
- Metric views can be certified for increased trust and reliability
- Go to the Mteric View Overview page
- Set a certification (Edit icon below the Metric View name)

# Create a Genie space with Metric View

Now, we will create a Genie Space with the metric view.

1. Go to the metric view Overview in Catalog Explorer.
2. Click the `Create` button in the top right corner.
3. Click `Genie Space`.
4. The Metric View will be auto-selected. Click `Create`.
5. [Optional] Instead of creating the Genie Space from the metric view Overview page, you can go to `Genie` on the left menu and create a Genie Space after selecting the Metric View that was previously created.
6. Start interacting with Genie. Ask questions in natural language and Genie will give back results with the generated query and results from the metric view.

### Some sample questions

- Start by clicking on `Explain the data set`
- Show Average Age Group by Gender
- What is the active user distribution by canal?
- What is the churn rate by age group?
- What is the total order amount by gender?



# Create a Dashboard with Metric View

We will now create a Dashboard with the metric view created in the previous steps.

1. Go to the metric view Overview in Catalog Explorer.
2. Click the `Create` button in the top right corner.
3. Click `Dashboard`.
4. [Optional] Instead of creating the Dashboard from the metric view Overview page, you can go to `Dashboards` on the left menu and create one. Go to the `Data` tab and select the Metric View that was previously created.
5. Start creating visualizations in the dashboard.

## Use the Dashboard AI assistant to create a visualization
1. Click on the icon to `Add a visualization`.
2. Place the box anywhere on the page.
3. In the textbox to `Ask the assistant to create a chart...` give the following input
~~~
Active Users by Age Group
~~~
4. Hit the Enter button or click on `Submit`.
5. The assistant will create a visualization. Check the widget values in the right pane and click `Accept.`


## Manually create a visualization
1. Click the icon to `Add a visualization`.
2. Place the box anywhere on the page.
3. On the right pane, under `Visualization`, select `Pie`.
4. Select `Churn Rate` for `Angle`. Notice that MEASURE is automatically added.
5. Under `Color`, select `Canal`.
6. This will display the pie chart for Churn Rate by Canal.
7. [Optional] Check `Title` under widget and add `Churn Rate by Canal` as title in the visualization.

### Exercise

- Add some more visualizations in the dashboard
- Add filters to the dashboard
- Add a name to the dashboard
- Save the dashboard
- Publish the dashboard

# Use SQL to query the metric view

Next, we will see how to interact with a metric view using SQL.

## List all views in the schema
Let's see how to find the metric views programmatically.

In [0]:
SHOW VIEWS;

> **Note:** isMetric is true for metric views.

## Describe a metric view
Let's look at the metric view.

In [0]:
describe table extended churn_users_metric_view;

> **Note:** Type is METRIC_VIEW and 'View Text' contains the entire YAML.

## Write a query using the metric view

### Exercise

- Write a query to get Total Users per Age Group from `churn_users_metric_view`.
- Some examples below show how to query a metric view using SQL.

> **Hint:** Remember to use the MEASURE keyword.

# Advantages of using Metric View

#### Example 1: Business Language Transformation
✅ No more **complicated logic** for defined dimensions.  
✅ **Business-friendly dimension** names that everyone understands.  
✅ The **translations happen once** in the Metric View definition, not in every query.  


In [0]:
SELECT
  `Gender`,
  MEASURE(`Active Users`)
FROM
  churn_users_metric_view
GROUP BY
  ALL;

#### Example 2: Time-Based Analysis
✅ Complicated **Window functions** are no longer needed; just selecting the measure is sufficient.  
✅ No need for analysts to **remember and repeat** complex logic.  
✅ Different teams will all use the **same translations**.  

In [0]:
SELECT
  `Last Activity Date`,
  MEASURE(`Trailing 30-Day Active Users`)
FROM
  churn_users_metric_view
GROUP BY ALL
ORDER BY `Last Activity Date` DESC;

# Next: Start building analysis with Databricks SQL

Now that we are ready for data analysis, let's see how our Data Analyst team can leverage them to run BI workloads.

Jump to the [BI / Data warehousing notebook]($../03-AI-BI-data-warehousing/03.1-AI-BI-Datawarehousing) or [Go back to the introduction]($../00-churn-introduction-lakehouse)